#how to setup rtmp server, please reference below url
https://www.cnblogs.com/nowgood/p/ffmpegnginx.html

#send rtmp data from pi4 to server, can use follow command
ffmpeg -f v4l2 -r 45 -video_size vga -pix_fmt yuv420p12be -i /dev/video0 -b:v 500k -c:v h264_omx -preset ultrafast -an -f flv rtmp://ec2-54-222-130-39.cn-north-1.compute.amazonaws.com.cn/live/123456


In [ ]:
import boto3
from IPython.display import clear_output, Image, display, HTML
import numpy as np
import cv2
import base64
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook
import time
output_notebook()

In [ ]:
class VideoPlayer(object):
    def __init__(self):
        self._init = False
        self._myImage = None
        
    def __call__(self, frame):
        if frame is None:
            return
        if self._init is False:
            self.init_display(frame)
            self._init = True
        else:
            self.update_display(frame)

    def init_display(self, frame):
        assert frame is not None
        frame=cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA) # because Bokeh expects a RGBA image
        frame=cv2.flip(frame, -1) # because Bokeh flips vertically
        width=frame.shape[1]
        height=frame.shape[0]
        p = figure(x_range=(0,width), y_range=(0,height), output_backend="webgl", width=width, height=height)
        self._myImage = p.image_rgba(image=[frame], x=0, y=0, dw=width, dh=height)
        show(p, notebook_handle=True)
    
    def update_display(self, frame):
        assert frame is not None
        frame=cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
        frame=cv2.flip(frame, -1)
        self._myImage.data_source.data['image']=[frame]
        push_notebook()

In [ ]:
"""
need add rtmp support for sagemaker, please follow this page
https://answers.opencv.org/question/180776/build-opencv-with-ffmpeg-support/
"""
url="rtmp://ec2-54-222-130-39.cn-north-1.compute.amazonaws.com.cn/live/123456"

vcap = cv2.VideoCapture(url)
player = VideoPlayer()

ret, frame = vcap.read()

while(True):
    # Capture frame-by-frame
    ret, frame = vcap.read()

    if frame is not None:
        # Display the resulting frame
        #cv2.imshow('frame',frame)
        player(frame)
        #time.sleep(0.1)

        # Press q to close the video windows before it ends if you want
        #if cv2.waitKey(22) & 0xFF == ord('q'):
        #    break
    else:
        print("Frame is None")
        break

# When everything done, release the capture
vcap.release()
print("Video stop")